In [8]:
import requests
from bs4 import BeautifulSoup
import json

In [2]:
site_map = "https://www.bankofabyssinia.com/faq/"

In [3]:
response = requests.get(site_map)

In [4]:
response

<Response [200]>

In [5]:
html = response.text

In [6]:
soup = BeautifulSoup(html, "html.parser")

In [7]:
faq_categories = {
    "Interest-Free Banking": "faq-accordion-1",
    "Online Banking": "faq-accordion-2",
    "Card Banking": "faq-accordion-3"
}

In [9]:
def scrape_faqs_to_dict():
    llm_training_data = []
    
    for category, container_id in faq_categories.items():
        container = soup.find('div', id=container_id)
        if not container:
            print(f"Warning: Container {container_id} not found for {category}")
            continue
            
        for item in container.find_all('div', class_='faq-main'):
            
            question = item.find('div', class_='faq-title').h4.get_text(strip=True)
            answer = item.find('div', class_='faq-content').get_text(strip=True, separator=' ')
            
            
            answer = ' '.join(answer.split())
            
            
            llm_training_data.append({
                "instruction": "Answer the following banking FAQ question",  
                "input": question,
                "output": answer,
                "category": category,
                "source": "Bank of Abyssinia Official FAQ",
                "language": "English"
            })
    
    return llm_training_data

In [10]:
faq_data = scrape_faqs_to_dict()

with open("boa_faqs_llm_training.jsonl", "w", encoding="utf-8") as f:
    for item in faq_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print(f"Scraped {len(faq_data)} FAQs. Sample:")
print(json.dumps(faq_data[0], indent=2, ensure_ascii=False))

Scraped 43 FAQs. Sample:
{
  "instruction": "Answer the following banking FAQ question",
  "input": "How much will I pay for the service?",
  "output": "There are no charges to register for Internet Banking.",
  "category": "Interest-Free Banking",
  "source": "Bank of Abyssinia Official FAQ",
  "language": "English"
}


In [11]:
faq_data

[{'instruction': 'Answer the following banking FAQ question',
  'input': 'How much will I pay for the service?',
  'output': 'There are no charges to register for Internet Banking.',
  'category': 'Interest-Free Banking',
  'source': 'Bank of Abyssinia Official FAQ',
  'language': 'English'},
 {'instruction': 'Answer the following banking FAQ question',
  'input': 'What is the currency to open IFB Diaspora account?',
  'output': 'Customers can open accounts using: US Dollar, Pound Sterling and/or Euro.',
  'category': 'Interest-Free Banking',
  'source': 'Bank of Abyssinia Official FAQ',
  'language': 'English'},
 {'instruction': 'Answer the following banking FAQ question',
  'input': 'What document should I fulfill to borrow Diaspora Finance?',
  'output': 'A Diaspora saving account Copy of valid passport Letter of undertaking to repay the loan in foreign currency Ethiopian Original ID card (Yellow Card copy) A letter that confirms monthly salary The authenticated sales contract agree